<a href="https://colab.research.google.com/github/MariaMacedooo/Maria-Macedo/blob/main/C%C3%B3digo%20Garcia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

# Baixar stopwords do NLTK (somente na primeira execução)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# ⚠️ Carregue o arquivo CSV no Google Colab (menu lateral > arquivos > upload)
# Em seguida, certifique-se de que o nome do arquivo esteja correto abaixo:
df = pd.read_csv("test_recipes_single_column.csv")

# Verifique se a coluna esperada existe
if 'A' not in df.columns:
    raise KeyError("O arquivo deve conter uma coluna chamada 'A' com os dados consolidados.")

# Preprocessamento do texto: limpeza e remoção de stopwords
def preprocess_text(text):
    tokens = str(text).lower().split()
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Aplicar pré-processamento
df['processed'] = df['A'].apply(preprocess_text)

# Separar nome da receita (assumindo que o nome está logo após o primeiro '|')
df['Name'] = df['A'].apply(lambda x: str(x).split('|')[1].strip() if '|' in str(x) else 'Unknown')

# Função de recomendação por similaridade de cosseno
def recomendar_por_descricao(nome_receita, top_n=5):
    if nome_receita not in df['Name'].values:
        return f"❌ Receita '{nome_receita}' não encontrada no dataset."

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['processed'])

    idx = df[df['Name'] == nome_receita].index[0]
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()

    similar_indices = cosine_sim.argsort()[-top_n-1:-1][::-1]
    for i in similar_indices:
      print(f"{df.iloc[i]['Name']} - similaridade: {cosine_sim[i]:.2f}")
# Entrada do usuário
nome_busca = input("\nDigite o nome da receita para buscar recomendações: ").strip()

# Resultado
print(f"\n🍽️ Receitas semelhantes a '{nome_busca}':\n")
recomendar_por_descricao(nome_busca)